In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from rinet.metrics import norm_err, bhattacharyya_gaussian_distance
import helpers as lib
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))
from common.helpers import bootstrap_paired_diff

# ---- constants -----
data_dir = '../../data/'
predictions_path = './predictions/'
errors_path = './errors/'
accuracy_threshold_ne = 0.1
accuracy_threshold_bc = 0.98

datasets = {
    'liver_1d_ornone': f"{data_dir}/liver/outlier_removal_none/1d/",
    'liver_1d_orsample': f"{data_dir}/liver/outlier_removal_samplewise/1d/",
    'liver_1d_orpanel': f"{data_dir}/liver/outlier_removal_panelwise/1d/",
    'liver_2d_ornone': f"{data_dir}/liver/outlier_removal_none/2d/",
    'liver_2d_orsample': f"{data_dir}/liver/outlier_removal_samplewise/2d/",
    'liver_2d_orpanel': f"{data_dir}/liver/outlier_removal_panelwise/2d/",
    'simulated_1d': f"{data_dir}/simulated/simulated_1d/test/",
    'simulated_2d': f"{data_dir}/simulated/simulated_2d/test/"
}

os.makedirs(errors_path, exist_ok=True)

analytes_to_evaluate = [
    'alanine aminotransferase',
    'albumin',
    'alkaline phosphatase',
    'aspartate aminotransferase',
    'bilirubin',
    'cholesterol',
    'cholinesterase',
    'creatinine',
    'gamma-glutamyl transferase',
    'total protein'
]


In [2]:
# liver 1d

# set parameters
# dataset = 'liver_1d_orsample'  # dataset key
models = [  # model keys
    'rinet_v1',
    'rinet_v1_log',
    'rinet_v2',
    'gmm',
    'reflimR',
    'refineR',
]

for dataset in ['liver_1d_ornone']:  # 'liver_1d_orsample', 'liver_1d_orpanel']:
    # compute errors
    targets = lib.load_pickle(f"{datasets[dataset]}/targets.pkl")
    metadata = pd.read_csv(f"{datasets[dataset]}/metadata.csv", index_col=0)
    metadata['group_key'], _ = pd.factorize(pd.Series(list(zip(metadata['analyte'], metadata['gender']))))
    
    # build mask for allowed analytes
    mask = metadata['analyte'].isin(analytes_to_evaluate)
    
    errors = {}
    for model in models:
        p = lib.load_pickle(f"{predictions_path}/{model}_{dataset}.pkl")
    
        # zip only rows where mask is True
        errors[model] = np.array([
            norm_err(i, j) if (m and ~np.any(np.isnan(j))) else np.nan
            for i, j, m in zip(targets, p[0], mask)
        ])
    lib.save_pickle(errors, f"{errors_path}/{dataset}.pkl")
    
    # prepare statistics
    stats = {}
    for method, values in errors.items():
        stats[method] = [
            np.nanmean(values),                  # mean
            np.nanmedian(values),                # median
            np.nanmean(values <= accuracy_threshold_ne) * 100  # % under threshold
        ]
    
    # create DataFrame
    df = pd.DataFrame(stats, index=['mean', 'median', '% under threshold'])
    print(df)
    print(bootstrap_paired_diff(errors))


                    rinet_v1  rinet_v1_log   rinet_v2        gmm    reflimR  \
mean                0.291842      0.103670   0.060919   0.110260   0.092046   
median              0.111933      0.084863   0.046599   0.083424   0.072015   
% under threshold  45.000000     60.000000  90.000000  55.000000  60.000000   

                     refineR  
mean                0.108132  
median              0.090741  
% under threshold  65.000000  
        method_1      method_2  stat_diff  ci_lower  ci_upper  significant
0       rinet_v1  rinet_v1_log   0.188172  0.071334  0.333247         True
1       rinet_v1      rinet_v2   0.230923  0.092960  0.394791         True
2       rinet_v1           gmm   0.181582  0.049397  0.344790         True
3       rinet_v1       reflimR   0.199796  0.068254  0.364906         True
4       rinet_v1       refineR   0.183711  0.060017  0.350240         True
5   rinet_v1_log      rinet_v2   0.042751  0.010968  0.075359         True
6   rinet_v1_log           gmm  -0

In [3]:
from scipy.stats import skew
skews = []
data = lib.load_pickle(f"{datasets['liver_1d_ornone']}/samples.pkl")
for i in data:
    skews.append(skew(i))
    
df = pd.DataFrame(errors)
df['analyte'] = metadata['analyte']
df['gender'] = metadata['gender']
df = df[list(df.columns[-2:]) + list(df.columns[:-2])]
df['skew'] = skews
df


,analyte,gender,rinet_v1,rinet_v1_log,rinet_v2,gmm,reflimR,refineR,skew
0,alanine aminotransferase,M,0.090891,0.062674,0.063089,0.052962,0.054563,0.092312,4.139540
1,alanine aminotransferase,F,0.099210,0.086146,0.068678,0.031978,0.107995,0.163999,8.105183
2,albumin,M,0.057305,0.024340,0.026686,0.005252,0.042281,0.072928,-0.232862
3,albumin,F,0.123246,0.092884,0.042881,0.047803,0.064402,0.073593,-0.007484
4,alkaline phosphatase,M,0.068434,0.047664,0.071055,0.095574,0.132238,0.090318,0.506994
5,alkaline phosphatase,F,0.095615,0.083579,0.025936,0.204426,0.101951,0.091164,5.897462
6,aspartate aminotransferase,M,1.150369,0.215761,0.070330,0.027292,0.026163,0.084360,4.666552
7,aspartate aminotransferase,F,1.103046,0.284478,0.038314,0.114271,0.074759,0.079127,5.642718
8,bilirubin,M,0.553276,0.061998,0.002425,0.143849,0.220882,0.250258,7.313790
9,bilirubin,F,0.388000,0.146032,0.012058,0.115797,0.019868,0.029927,10.785410


In [4]:
del dataset, targets, errors, model, p, stats, method, values, df  # clean up


In [5]:
# liver 2d
import ast

# set parameters
# dataset = 'liver_2d'  # dataset key
models = [  # model keys
    'rinet_v2',
    'gmm',
]

for dataset in ['liver_2d_ornone']:  # 'liver_2d_orsample', 'liver_2d_orpanel']:
    # compute errors
    targets = lib.load_pickle(f"{datasets[dataset]}/targets.pkl")
    metadata = pd.read_csv(f"{datasets[dataset]}/metadata.csv", index_col=0)
    metadata['group_key'], _ = pd.factorize(
        pd.Series(list(zip(metadata['analyte_pair'], metadata['gender'])))
    )
    
    # Convert the stringified tuple to an actual tuple, then check both elements
    mask = metadata['analyte_pair'].apply(
        lambda s: all(
            analyte.strip() in analytes_to_evaluate
            for analyte in ast.literal_eval(s)
        )
    )
    
    errors = {}
    for model in models:
        p = lib.load_pickle(f"{predictions_path}/{model}_{dataset}.pkl")
        errors[model] = np.array([
            np.exp(-bhattacharyya_gaussian_distance(i[0], i[1], j['mean'], j['covariance']))
            if m else np.nan
            for i, j, m in zip(targets, p[1], mask)
        ])
    
    lib.save_pickle(errors, f"{errors_path}/{dataset}.pkl")
    
    # prepare statistics
    stats = {}
    for method, values in errors.items():
        stats[method] = [
            np.nanmean(values),                  # mean
            np.nanmedian(values),                # median
            np.nanmean(values >= accuracy_threshold_bc) * 100  # % above threshold
        ]
    
    # create DataFrame
    df = pd.DataFrame(stats, index=['mean', 'median', '% under threshold'])
    print(df)
    print(bootstrap_paired_diff(errors))
    del dataset, targets, errors, model, p, stats, method, values, df  # clean up


                    rinet_v2        gmm
mean                0.991156   0.985262
median              0.993027   0.994529
% under threshold  90.000000  83.333333
   method_1 method_2  stat_diff  ci_lower  ci_upper  significant
0  rinet_v2      gmm   0.005894  0.001264  0.011144         True


In [6]:
# simulated 1d

# set parameters
dataset = 'simulated_1d'  # dataset key
models = [  # model keys
    'rinet_v1',
    'rinet_v2',
    'gmm',
    'refineR',
    'reflimR'
]

# compute errors

# compute bhattacharyya distance errors for rinet_v2 and gmm for
# trend analysis combined with simulated_2d
targets, _ = lib.get_targets_simulated(f"{datasets[dataset]}/")  # load targets
targets = [[i[0], np.array([i[1]**2])] for i in targets]  # convert 1d std to covariance
errors = {}
for model in ['rinet_v2', 'gmm']:
    p = lib.load_pickle(f"{predictions_path}/{model}_{dataset}.pkl")
    errors[model] = np.array([
        bhattacharyya_gaussian_distance(i[0], i[1], j['mean'], j['covariance']) for i, j in zip(targets, p[1])
    ])
lib.save_pickle(errors, f"{errors_path}/{dataset}_bd.pkl")

# now compute norm_err
_, targets = lib.get_targets_simulated(f"{datasets[dataset]}/")  # load targets
errors = {}
for model in models:
    p = lib.load_pickle(f"{predictions_path}/{model}_{dataset}.pkl")
    errors[model] = np.array([
        norm_err(i, j) if ~np.any(np.isnan(j)) else np.nan
        for i, j in zip(targets, p[0])
    ])
lib.save_pickle(errors, f"{errors_path}/{dataset}.pkl")

# prepare statistics
stats = {}
for method, values in errors.items():
    stats[method] = [
        np.nanmean(values),                            # mean
        np.nanmedian(values),                          # median
        np.nanmean(values <= accuracy_threshold_ne) * 100  # % under threshold
    ]

# create DataFrame
df = pd.DataFrame(stats, index=['mean', 'median', '% under threshold'])
print(df)
print(bootstrap_paired_diff(errors))
del dataset, models, targets, errors, model, p, stats, method, values, df  # clean up


                    rinet_v1   rinet_v2        gmm    refineR    reflimR
mean                0.082731   0.054982   0.084003   0.080354   0.096752
median              0.064811   0.042803   0.058924   0.052187   0.072344
% under threshold  70.800000  87.600000  71.200000  75.000000  63.500000
   method_1  method_2  stat_diff  ci_lower  ci_upper  significant
0  rinet_v1  rinet_v2   0.027749  0.024217  0.031505         True
1  rinet_v1       gmm  -0.001272 -0.007246  0.005160        False
2  rinet_v1   refineR   0.002271 -0.003498  0.007732        False
3  rinet_v1   reflimR  -0.014020 -0.019500 -0.008557         True
4  rinet_v2       gmm  -0.029021 -0.034409 -0.023917         True
5  rinet_v2   refineR  -0.025348 -0.030613 -0.020346         True
6  rinet_v2   reflimR  -0.041769 -0.046498 -0.036497         True
7       gmm   refineR   0.003726 -0.002409  0.010265        False
8       gmm   reflimR  -0.012749 -0.018830 -0.006295         True
9   refineR   reflimR  -0.016451 -0.022464 -0.01

In [7]:
# simulated 2d

# set parameters
dataset = 'simulated_2d'  # dataset key
models = [  # model keys
    'rinet_v2',
    'gmm',
]

# compute errors
targets = lib.get_targets_simulated(f"{datasets[dataset]}/")  # load targets
errors = {}
for model in models:
    p = lib.load_pickle(f"{predictions_path}/{model}_{dataset}.pkl")
    errors[model] = np.array([
        np.exp(-bhattacharyya_gaussian_distance(i[0], i[1], j['mean'], j['covariance'])) for i, j in zip(targets, p[1])
    ])
lib.save_pickle(errors, f"{errors_path}/{dataset}.pkl")

# prepare statistics
stats = {}
for method, values in errors.items():
    stats[method] = [
        np.nanmean(values),                            # mean
        np.nanmedian(values),                          # median
        np.nanmean(values >= accuracy_threshold_bc) * 100  # % above threshold
    ]

# create DataFrame
df = pd.DataFrame(stats, index=['mean', 'median', '% under threshold'])
print(df)
print(bootstrap_paired_diff(errors))
del dataset, models, targets, errors, model, p, stats, method, values, df  # clean up


                    rinet_v2        gmm
mean                0.984777   0.975493
median              0.992751   0.991046
% under threshold  78.100000  68.800000
   method_1 method_2  stat_diff  ci_lower  ci_upper  significant
0  rinet_v2      gmm   0.009285  0.006767  0.011971         True
